In [ ]:
# # Use this function to conig notebook with python. Use it only once for per script.
# import os
# import sys
# def config_notebook():
#   current_dir = os.path.dirname(os.path.abspath("__file__"))
#   sys.path.append(current_dir)
#   print(sys.path)
# config_notebook()

In [108]:
import json
def read_json_data(path):
  try:
    with open(path, 'r') as file:
      return json.load(file)
  except Exception as e:
    print(f"Failed reading '{path}': {e}")


In [160]:
from db_connection import get_client
from utils import *

In [163]:
printer = get_printer()
client = get_client()
production_db = client.production

author_col_name = 'author'
author_data_path = '../data/author_data.json'
author_validator_path = '../schemas/author_schema.json'

book_col_name = 'book'
book_data_path = '../data/book_data.json'
book_validatator_path = '../schemas/book_schema.json'

Connection to MongoDB is succesful!


In [151]:
author_validator = read_json_data(author_validator_path)
book_validator = read_json_data(book_validatator_path)

production_db.create_collection(author_col_name, validator=author_validator)
production_db.create_collection(book_col_name, validator=book_validator)

Collection(Database(MongoClient(host=['cluster0-shard-00-02.f5fvl.mongodb.net:27017', 'cluster0-shard-00-01.f5fvl.mongodb.net:27017', 'cluster0-shard-00-00.f5fvl.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', authmechanism='SCRAM-SHA-1', replicaset='atlas-gicwct-shard-0', tls=True), 'production'), 'book')

In [152]:
author_data = insert_json_data(production_db[author_col_name], author_data_path)
book_data = insert_json_data(production_db[book_col_name], book_data_path)

Inserted document amount:  10
Inserted document IDs: [ObjectId('67853b61b909ea751b6406b8'), ObjectId('67853b61b909ea751b6406b9'), ObjectId('67853b61b909ea751b6406ba'), ObjectId('67853b61b909ea751b6406bb'), ObjectId('67853b61b909ea751b6406bc'), ObjectId('67853b61b909ea751b6406bd'), ObjectId('67853b61b909ea751b6406be'), ObjectId('67853b61b909ea751b6406bf'), ObjectId('67853b61b909ea751b6406c0'), ObjectId('67853b61b909ea751b6406c1')]
Inserted document amount:  10
Inserted document IDs: [ObjectId('67853b61b909ea751b6406c2'), ObjectId('67853b61b909ea751b6406c3'), ObjectId('67853b61b909ea751b6406c4'), ObjectId('67853b61b909ea751b6406c5'), ObjectId('67853b61b909ea751b6406c6'), ObjectId('67853b61b909ea751b6406c7'), ObjectId('67853b61b909ea751b6406c8'), ObjectId('67853b61b909ea751b6406c9'), ObjectId('67853b61b909ea751b6406ca'), ObjectId('67853b61b909ea751b6406cb')]


In [159]:
authors_and_books_pipeline = [
  {
    "$lookup": {
      "from": "book",
      "localField": "_id",
      "foreignField": "authors",
      "as": "books"
    }
  },
  {
    "$limit": 3
  }
]
aggregate(production_db[author_col_name], authors_and_books_pipeline)

[{'_id': ObjectId('67853b61b909ea751b6406b8'),
  'books': [],
  'date_of_birth': '1987-05-14',
  'first_name': 'Sophia',
  'last_name': 'Anderson'},
 {'_id': ObjectId('67853b61b909ea751b6406b9'),
  'books': [],
  'date_of_birth': '1991-09-23',
  'first_name': 'Liam',
  'last_name': 'White'},
 {'_id': ObjectId('67853b61b909ea751b6406ba'),
  'books': [],
  'date_of_birth': '1994-02-12',
  'first_name': 'Olivia',
  'last_name': 'Harris'}]


In [162]:
authors_book_count_pipeline = [
	{
		"$lookup": {
      "from": "book",
			"localField": "_id",
			"foreignField": "authors",
			"as": "books"
      }
	},
	{
		"$addFields": {
			"total_books": {"$size": "$books"}
      }
	},
	{
		"$project": {
      "first_name": 1,
			"last_name": 1,
			"total_books": 1,
			"_id": 0
      }
	}
]
aggregate(production_db[author_col_name], authors_and_books_pipeline)

[{'_id': ObjectId('67853b61b909ea751b6406b8'),
  'books': [],
  'date_of_birth': '1987-05-14',
  'first_name': 'Sophia',
  'last_name': 'Anderson'},
 {'_id': ObjectId('67853b61b909ea751b6406b9'),
  'books': [],
  'date_of_birth': '1991-09-23',
  'first_name': 'Liam',
  'last_name': 'White'},
 {'_id': ObjectId('67853b61b909ea751b6406ba'),
  'books': [],
  'date_of_birth': '1994-02-12',
  'first_name': 'Olivia',
  'last_name': 'Harris'}]


In [ ]:
patch_all()

schema_author = Schema({"_id": ObjectId,
                 "first_name": pyarrow.string(),
                 "last_name": pyarrow.string(),
                 "date_of_birth": dt})

df = production.author.find_pandas_all({},schema=schema_author)
arrow_table = production.author.find_arrow_all({}, schema=schema_author)
ndarrays = production.author.find_numpy_all({}, schema=schema_author)
print(ndarrays)
